In [ ]:
"""
Kaggle Downloader Module
=========================
Descarga automática de datasets desde Kaggle.
"""

import os
from pathlib import Path
from typing import Optional, Union
import zipfile
import shutil

try:
    from kaggle.api.kaggle_api_extended import KaggleApi
    KAGGLE_AVAILABLE = True
except ImportError:
    KAGGLE_AVAILABLE = False
    print("⚠️ Kaggle API no instalada. Instala con: pip install kaggle")


class KaggleDownloader:
    """
    Descargador automático de datasets de Kaggle

    Requisitos:
    1. Instalar: pip install kaggle
    2. Crear cuenta en Kaggle
    3. Descargar API token desde: https://www.kaggle.com/settings
    4. Colocar kaggle.json en: ~/.kaggle/kaggle.json (Linux/Mac) o C:\\Users\\<usuario>\\.kaggle\\kaggle.json (Windows)
    """

    def __init__(self, download_dir: Optional[Union[str, Path]] = None):
        """
        Inicializa el descargador

        Args:
            download_dir: Directorio donde descargar (default: data/raw)
        """
        if not KAGGLE_AVAILABLE:
            raise ImportError(
                "Kaggle API no está instalada. "
                "Instala con: pip install kaggle"
            )

        self.api = KaggleApi()
        self.api.authenticate()

        if download_dir is None:
            from src.config import config
            download_dir = config.RAW_DATA_DIR

        self.download_dir = Path(download_dir)
        self.download_dir.mkdir(parents=True, exist_ok=True)

        print("✅ Kaggle API autenticada correctamente")

    def download_dataset(
        self,
        dataset_identifier: str,
        force: bool = False,
        unzip: bool = True
    ) -> Path:
        """
        Descarga un dataset de Kaggle

        Args:
            dataset_identifier: ID del dataset (ej: 'carrie1/ecommerce-data')
            force: Forzar descarga aunque ya exista
            unzip: Descomprimir automáticamente

        Returns:
            Path: Ruta donde se descargó el dataset

        Ejemplo:
            >>> downloader = KaggleDownloader()
            >>> path = downloader.download_dataset('carrie1/ecommerce-data')
        """
        print(f"\n📥 Descargando dataset: {dataset_identifier}")

        # Verificar si ya existe
        dataset_name = dataset_identifier.split('/')[-1]
        target_dir = self.download_dir / dataset_name

        if target_dir.exists() and not force:
            print(f"✅ Dataset ya existe en: {target_dir}")
            return target_dir

        # Descargar
        try:
            self.api.dataset_download_files(
                dataset_identifier,
                path=self.download_dir,
                unzip=unzip
            )

            print(f"✅ Dataset descargado en: {self.download_dir}")

            # Si se descomprimió, mover archivos a carpeta del dataset
            if unzip:
                self._organize_files(dataset_name)

            return target_dir

        except Exception as e:
            print(f"❌ Error descargando dataset: {e}")
            raise

    def download_competition_data(
        self,
        competition_name: str,
        force: bool = False
    ) -> Path:
        """
        Descarga datos de una competencia de Kaggle

        Args:
            competition_name: Nombre de la competencia (ej: 'titanic')
            force: Forzar descarga

        Returns:
            Path: Ruta donde se descargó
        """
        print(f"\n🏆 Descargando competencia: {competition_name}")

        target_dir = self.download_dir / competition_name

        if target_dir.exists() and not force:
            print(f"✅ Competencia ya existe en: {target_dir}")
            return target_dir

        target_dir.mkdir(exist_ok=True)

        try:
            self.api.competition_download_files(
                competition_name,
                path=target_dir
            )

            # Descomprimir archivos zip
            for zip_file in target_dir.glob('*.zip'):
                with zipfile.ZipFile(zip_file, 'r') as zip_ref:
                    zip_ref.extractall(target_dir)
                zip_file.unlink()  # Eliminar zip después de extraer

            print(f"✅ Competencia descargada en: {target_dir}")
            return target_dir

        except Exception as e:
            print(f"❌ Error descargando competencia: {e}")
            raise

    def list_dataset_files(self, dataset_identifier: str):
        """Lista archivos disponibles en un dataset"""
        print(f"\n📋 Archivos en dataset '{dataset_identifier}':")
        files = self.api.dataset_list_files(dataset_identifier).files

        for file in files:
            print(f"   • {file.name} ({file.size} bytes)")

        return [file.name for file in files]

    def search_datasets(self, query: str, max_results: int = 10):
        """
        Busca datasets en Kaggle

        Args:
            query: Término de búsqueda
            max_results: Número máximo de resultados
        """
        print(f"\n🔍 Buscando datasets: '{query}'")

        datasets = self.api.dataset_list(search=query, page_size=max_results)

        print(f"\n📊 Resultados ({len(datasets)}):")
        for i, dataset in enumerate(datasets, 1):
            print(f"\n{i}. {dataset.ref}")
            print(f"   Título: {dataset.title}")
            print(f"   Tamaño: {dataset.size}")
            print(f"   Última actualización: {dataset.lastUpdated}")

        return datasets

    def _organize_files(self, dataset_name: str):
        """Organiza archivos descargados en una carpeta"""
        # Buscar archivos en download_dir que correspondan al dataset
        target_dir = self.download_dir / dataset_name
        target_dir.mkdir(exist_ok=True)

        # Mover archivos CSV/Excel al directorio del dataset
        for file in self.download_dir.glob('*'):
            if file.is_file() and file.suffix in ['.csv', '.xlsx', '.json', '.txt']:
                shutil.move(str(file), target_dir / file.name)


# Funciones de conveniencia
def download_from_kaggle(
    dataset_identifier: str,
    download_dir: Optional[Union[str, Path]] = None
) -> Path:
    """
    Función rápida para descargar un dataset

    Ejemplo:
        >>> path = download_from_kaggle('carrie1/ecommerce-data')
    """
    downloader = KaggleDownloader(download_dir)
    return downloader.download_dataset(dataset_identifier)


def setup_kaggle_credentials():
    """
    Guía interactiva para configurar credenciales de Kaggle
    """
    print("""
    📋 CONFIGURACIÓN DE KAGGLE API
    ==============================

    1. Ve a: https://www.kaggle.com/settings
    2. En la sección "API", haz clic en "Create New API Token"
    3. Se descargará un archivo kaggle.json
    4. Colócalo en:
       • Windows: C:\\Users\\<tu-usuario>\\.kaggle\\kaggle.json
       • Linux/Mac: ~/.kaggle/kaggle.json
    5. En Linux/Mac, ejecuta: chmod 600 ~/.kaggle/kaggle.json

    ¿Necesitas ayuda? Visita: https://www.kaggle.com/docs/api
    """)

    kaggle_dir = Path.home() / '.kaggle'
    kaggle_file = kaggle_dir / 'kaggle.json'

    if kaggle_file.exists():
        print(f"✅ Archivo kaggle.json encontrado en: {kaggle_file}")
    else:
        print(f"⚠️ Archivo kaggle.json NO encontrado en: {kaggle_file}")
        print("   Sigue los pasos anteriores para configurarlo.")


__all__ = ['KaggleDownloader', 'download_from_kaggle', 'setup_kaggle_credentials']